In [11]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
import statsmodels 
import seaborn as sns
import scipy.stats as ss
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts 
import sklearn
import arch
import datetime as dt


from pmdarima.arima import auto_arima
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch
from statsmodels.stats.stattools import jarque_bera
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from tqdm import tqdm
from time import sleep
import warnings
warnings.filterwarnings('ignore', category=Warning)
sns.set()

In [12]:
def read_csv(name:str):
    df = pd.read_csv(f'../data/processed/{name}.csv')
    df.Date = pd.to_datetime(df.Date)
    df.set_index('Date', inplace = True)
    df.asfreq('12H')
    df.dropna(inplace = True)
    return df

In [13]:
df= read_csv('data_processed')
df_test = read_csv('data_test')

In [14]:
df.head()

,BTCUSDT,ETHUSDT,ADAUSDT,BNBUSDT,Ret_BTCUSDT,Ret_cum_BTCUSDT,Norm_BTCUSDT,Ret_ETHUSDT,Ret_cum_ETHUSDT,Norm_ETHUSDT,Ret_ADAUSDT,Ret_cum_ADAUSDT,Norm_ADAUSDT,Ret_BNBUSDT,Ret_cum_BNBUSDT,Norm_BNBUSDT
Date,,,,,,,,,,,,,,,,
2018-06-01 12:00:00,7521.01,579.00,0.22038,14.2888,2.168340,2.168340,102.168340,1.466800,1.466800,101.466800,2.122335,2.122335,102.122335,3.692308,3.692308,103.692308
2018-06-02 00:00:00,7652.28,592.73,0.22428,14.5000,1.745377,3.913718,103.951563,2.371330,3.838130,103.872912,1.769671,3.892006,103.929564,1.478081,5.170388,105.224964
2018-06-02 12:00:00,7640.03,590.85,0.22648,14.6732,-0.160083,3.753635,103.785155,-0.317176,3.520953,103.543452,0.980917,4.872923,104.949027,1.194483,6.364871,106.481858
2018-06-03 00:00:00,7714.85,619.93,0.23284,14.7861,0.979316,4.732950,104.801539,4.921723,8.442676,108.639574,2.808195,7.681118,107.896200,0.769430,7.134301,107.301161
2018-06-03 12:00:00,7714.26,619.66,0.22659,14.6995,-0.007648,4.725303,104.793524,-0.043553,8.399123,108.592258,-2.684247,4.996871,105.000000,-0.585685,6.548616,106.672714


In [15]:
def fit_auto(df, target, exogenous=None, exog=True, **kwargs):
    if exog:
        model = auto_arima(df[[f'Ret_{target}']], X=df[exogenous], 
                       max_order = None, max_p = 9, max_q = 9, max_d = 2, max_P = 6, max_Q = 6, max_D = 4,
                       maxiter = 70, trend = 'ct')
        print(model.summary())
    else:
        model = auto_arima(df[[f'Ret_{target}']], 
                       max_order = None, max_p = 9, max_q = 9, max_d = 2, max_P = 6, max_Q = 6, max_D = 4,
                       maxiter = 70, trend = 'ct')
        print(model.summary())
    
    return model




In [16]:
start_date = '2021-12-18 12:00:00'
end_date='2022-06-01 12:00:00'

def fit_pred(model, target, exogenous: list = None, start_date = start_date, end_date= end_date, df_test=df_test, exog=True):
    if exog:
        df_auto_pred = pd.DataFrame(
            model.predict(n_periods=len(df_test[start_date:end_date]), X=df_test[exogenous][start_date:end_date]),
            index=df_test[start_date:end_date].index)
    else:
        df_auto_pred = pd.DataFrame(
            model.predict(n_periods=len(df_test[start_date:end_date]), index=df_test[start_date:end_date].index))

    plt.figure(figsize=(20, 5))
    plt.plot(df_auto_pred, color="red", label="Auto Model Predictions")
    plt.plot(df_test[f'Ret_{target}'][start_date:end_date], color="blue", label="Real Data")
    plt.title(f"Auto Model Predictions vs Real Data for {target}", size=24)
    plt.xlabel("Date", size=16)
    plt.ylabel("Return", size=16)
    plt.legend(fontsize=14)
    plt.savefig(f'../reports/figures/{target}_prediction_vs_real.png')
    plt.show()


In [17]:
def analyze_residuals(model, name:str):
    residuals_df = pd.DataFrame(model.resid())
    print(residuals_df.describe())

    residuals = model.resid()
    lags = [1]
    # Perform Ljung-Box test
    lb_test = acorr_ljungbox(residuals, lags=lags, return_df=True)
    lb_p_value = lb_test.iloc[0, 1]

    # Perform Jarque-Bera test
    jb_stat, jb_p_value, skew, kurtosis = jarque_bera(residuals)

    # Perform heteroskedasticity test
    het_test = het_arch(residuals)
    het_p_value = het_test[1]

    # Perform ADF test
    adf_test = adfuller(residuals)
    adf_p_value = round(adf_test[1], 3)

    # Print summary of analysis
    print("Residual Analysis:")
    print("================================"*2)
    print(f"\nLjung-Box (lag 1) p-value: {lb_p_value}")
    print(f"\nJarque-Bera p-value: {jb_p_value}")
    print(f"\nHeteroskedasticity p-value: {het_p_value}")
    print(f"\nADF p-value: {adf_p_value}")
    print("================================"*2)
    
    print('\n')


    # Autocorrelation plot
    fig, ax = plt.subplots(figsize=(7, 4))
    plot_acf(residuals_df, lags=30, ax=ax)
    plt.title(f'Autocorrelation of Residuals_{name}')
    plt.xlabel('Lag')
    plt.ylabel('Autocorrelation')
    plt.savefig(f'../reports/figures/{name}_resid_acf.png')
    plt.show()

    # Partial autocorrelation plot
    fig, ax = plt.subplots(figsize=(7, 4))
    plot_pacf(residuals_df, lags=30, ax=ax)
    plt.title(f'Partial Autocorrelation of Residuals_{name}')
    plt.xlabel('Lag')
    plt.ylabel('Partial Autocorrelation')
    plt.savefig(f'../reports/figures/{name}_resid_pacf.png')
    plt.show()
    print('\n')
    model.plot_diagnostics(figsize=(20, 11))
    plt.savefig(f'../reports/figures/{name}_diagnostics.png')
    plt.show()



In [ ]:
model_arima_bnb = fit_auto(df, 'BNBUSDT', exog=False)

In [ ]:
analyze_residuals(model_arima_bnb, 'BNBUSDT')

In [ ]:
model_arima_bnb_exog = fit_auto(df, 'BNBUSDT', exogenous= ['Ret_BTCUSDT', 'Ret_ADAUSDT', 'Ret_ETHUSDT'])

In [ ]:
analyze_residuals(model_arima_bnb_exog, 'BNBUSDT_exog')

In [ ]:
fit_pred(model_arima_bnb_exog, 'BNBUSDT', exogenous= ['Ret_BTCUSDT', 'Ret_ADAUSDT', 'Ret_ETHUSDT'] )

In [ ]:
model_arima_btc = fit_auto(df, 'BTCUSDT', exog=False)


In [ ]:
analyze_residuals(model_arima_btc, 'BTCUSDT')

In [ ]:
model_arima_btc_exog = fit_auto(df, 'BTCUSDT', exogenous= ['Ret_BNBUSDT', "Ret_ETHUSDT", 'Ret_ADAUSDT'])


In [ ]:
analyze_residuals(model_arima_btc_exog, 'BTCUSDT_exog')

In [ ]:
fit_pred(model_arima_btc_exog, 'BTCUSDT', exogenous= ['Ret_BNBUSDT', "Ret_ETHUSDT", 'Ret_ADAUSDT'] )

In [ ]:


model_arima_ada = fit_auto(df, 'ADAUSDT', exog=False)


In [ ]:
analyze_residuals(model_arima_ada, 'ADAUSDT')

In [ ]:
model_arima_ada_exog = fit_auto(df, 'ADAUSDT', exogenous=['Ret_BTCUSDT', 'Ret_ETHUSDT', 'Ret_BNBUSDT'])


In [ ]:
analyze_residuals(model_arima_ada_exog, 'ADAUSDT_exog')

In [ ]:
fit_pred(model_arima_ada_exog, 'ADAUSDT', exogenous=['Ret_BTCUSDT', 'Ret_ETHUSDT', 'Ret_BNBUSDT'])

In [ ]:

model_arima_eth = fit_auto(df, 'ETHUSDT', exog=False)


In [ ]:
analyze_residuals(model_arima_eth, 'ETHUSDT')

In [ ]:
model_arima_eth_exog = fit_auto(df, 'ETHUSDT', exogenous=['Ret_BTCUSDT', 'Ret_BNBUSDT', 'Ret_ADAUSDT'])

In [ ]:
analyze_residuals(model_arima_eth_exog, 'ETHUSDT_exog')

In [ ]:
fit_pred(model_arima_eth_exog, 'ETHUSDT', exogenous=['Ret_BTCUSDT', 'Ret_BNBUSDT', 'Ret_ADAUSDT'] )

##### Forcasting Volitilty 

In [ ]:
df_test.head()[start_date:]

In [ ]:

df_com = pd.concat([df, df_test])

In [ ]:
df_com.tail()

In [ ]:
def garch_volatility(column, df = df_com, df_test = df_test, start_date = start_date):

    model = arch.arch_model(df[f'Ret_{column}'], vol='Garch', p=1, q=1)

    # Fit the model
    results = model.fit(last_obs=start_date,  update_freq=10)
    
    #forcast 
    pred_garch = results.forecast(horizon=1, align='target')


    # Plot the estimated volatility
    fig, ax = plt.subplots(figsize=(20, 6))
    pred_garch.residual_variance[start_date:].plot(figsize = (20,5), color = "red", zorder = 2, ax = ax)
    df_test[f'Ret_{column}'].abs().plot(color = "blue", zorder = 1, ax = ax)
    plt.title("Volatility Predictions", size = 24)    
    ax.set_xlabel('Time', fontsize = 15)
    ax.set_ylabel('Volatility', fontsize = 17)
    ax.set_title(f'GARCH Estimated Volatility {column}', fontsize = 20)
    ax.legend()
    plt.savefig(f'../reports/figures/Ret_{column}_garch_forcast.png')
    plt.show()


In [ ]:
garch_volatility('BNBUSDT')

In [ ]:
garch_volatility('ADAUSDT')

In [ ]:
garch_volatility('ETHUSDT')

In [ ]:
 garch_volatility('BTCUSDT')